<a href="https://colab.research.google.com/github/All4Nothing/hg-mldl/blob/main/5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치 

테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다

모델을 만들고 나서 마지막에 딱 한번만 사용하는 것이 좋다

## 검증 세트

**검증 세트 validation set** 훈련 세트 중 테스트에 사용하는 용도로 나뉜 세트

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다  
테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다

In [1]:

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [8]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

**교차 검증 cross validation**은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다

**k-폴드 교차 검증 k-fold cross validation**은 훈련 세트를 k 부분으로 나눠서 교차 검증을 수행하는 것  
보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용  

`cross_validate()` 교차 검증 함수. 평가할 모델 객체를 첫 번째 매개변수로 전달, 훈련 세트 전체를 전달한다.

`cross_val_score()`함수는 `cross_validate()` 함수의 결과 중에서 test_score 값만 반환한다.

In [11]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02191234, 0.01751471, 0.01075268, 0.01055026, 0.01233649]), 'score_time': array([0.00124216, 0.00132656, 0.0013063 , 0.00128508, 0.00140214]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


`cross_validate()`함수는 기본적으로 5-폴드 교차 검증을 수행한다  
`fit_time` : 모델 훈련 시간  
`score_time` : 모델 검증 시간  
`test_score` : 검증 폴드의 점수


교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다

In [13]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


`cross_validate()`는 훈련 세트를 섞어 폴드를 나누지 않는다  
교차 검증을 할 때 훈련 세트를 섞으려면 분할기 splitter를 지정해야 한다

`cross_validate()` 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다

In [14]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [16]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # n_splits 매개변수는 몇(k) 폴드 교차 검증을 할지 정한다
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

모델 파라미터 : 머신러닝 모델이 학습하는 파라미터  
하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

AutoML : 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술

**그리드 서치 Grid Search**를 사용하여 여러 매개변수를 동시에 바꿔가며 최적의 값을 찾아가는 도구

사이킷런의 `GridSearchCV` 클래스는 하이퍼파라미터 탐색과 교차 검증을 한번에 수행한다.

In [17]:
from sklearn.model_selection import  GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

`GridSearchCV`의 매개변수 기본값은 5 즉, `min_impurity_decrease` 값마다 5-폴드 교차 검증을 수행한다.  
많은 모델(5x5=25개)을 훈련하기 때문에 `n_jobs` 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정한다  
-1로 지정하면 시스템에 있는 모든 코어를 사용한다

In [19]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다  
사이킷런의 그리드 서치는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다  
이 모델은 gs 객체의 `best_estimator_` 속성에 저장되어 있다

In [20]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 `best_params` 속성에 저장되어 있다

In [22]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 `mean_test_score`키에 저장되어 있다

In [23]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [27]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 넘파이 arange() : 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다
          'max_depth': range(5, 20, 1), # 파이썬 range() : 정수만 사용할 수 있다
          'min_samples_split': range(2, 100, 10)
          }

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

**랜덤 서치 Random Search** 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵거나, 매개 변수 조건이 너무 많아 그리드 서치 수행 시간이 오래 걸릴 경우 사용하면 좋다

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다

In [31]:
from scipy.stats import uniform, randint

`uniform`과 `randint` 클래스 모두 균등 분포에서 샘플링 한다  
`randint` : 정수값 뽑음   
`uniform` : 실수값 뽑음

In [32]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 3, 7, 3, 7, 8, 2, 6, 5, 7])

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 84, 111,  91, 109, 103, 109,  98,  96, 104,  95]))

In [34]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.01620325, 0.9167649 , 0.65393695, 0.33496869, 0.84696733,
       0.33702295, 0.02412408, 0.96643553, 0.12333489, 0.63427106])

In [36]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
}

In [38]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcb338085d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcb325fc190>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcb325fc090>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcb325fc6d0>},
                   random_state=42)

In [39]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [40]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [41]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


테스트 세트 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적이다.